In [31]:
import argparse
import numpy as np
import os
import pandas as pd
import sys
import torch
import warnings
import lime
import shap
import torch.nn.functional as F
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

code_dir = os.path.split(os.getcwd())[0]
sys.path.append(code_dir)

base_dir = "/".join(code_dir.split('/')[:-1])
sys.path.append(f"{base_dir}/code/turing/examples-raw/gluesst_finetune/")
sys.path.append(f"{base_dir}/code/turing/src/")

from argparse import Namespace
from methods.bag_of_ngrams.processing import cleanReports, cleanSplit, stripChars
from pyfunctions.general import extractListFromDic, readJson
from pyfunctions.pathology import extract_synoptic, fixProstateLabels
from sklearn import preprocessing
from sklearn.metrics import f1_score
from turing.pathology.run_classifier import MODEL_CLASSES, processors, train_path as train
from methods.torch.processing import make_weights_for_balanced_classes
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from tqdm import tqdm, trange
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertForSequenceClassification
from turing.pathology.path_utils import evaluate, extract_features, load_tnlr_base, load_tnlr_tokenizer, path_dataset
from sklearn.model_selection import train_test_split
from lime.lime_text import LimeTextExplainer

args = {
    'model_type': 'tnlr', # tnlr, bert, medical_bert, pubmed_bert, biobert, clinical_biobert
    'cache_dir': f"{base_dir}/path_nlp_turing/output/fine_tuning/bert", # CHANGE
    'output_dir': f"{base_dir}/path_nlp_turing/output/fine_tuning/bert", # CHANGE
    'model_name_or_path': '',
    'task_name': 'sst-2',
    'config_name': '',
    'tokenizer_name': '',
    'do_train': True,
    'do_eval': True,
    'evaluate_during_training': True,
    'max_seq_length': 512,
    'do_lower_case': True,
    'per_gpu_train_batch_size': 8,
    'per_gpu_eval_batch_size': 8,
    'gradient_accumulation_steps': 1,
    'learning_rate': 7e-6,
    'weight_decay': 0.0,
    'adam_epsilon': 1e-8,
    'max_grad_norm': 1,
    'num_train_epochs': 3.0,
    'max_steps': -1,
    'warmup_ratio': 0.2,
    'logging_steps': 50,
    'eval_all_checkpoints': True,
    'no_cuda': False,
    'overwrite_output_dir': True,
    'seed': 42,
    'overwrite_cache': True,
    'metric_for_choose_best_checkpoint': None,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'local_rank': -1,
    'num_train_epochs': 25, 
    'n_gpu': 1,
    'device': 'cpu'
}

def fixLabel(data):
    for split_ in ['train', 'val', 'test', 'dev_test']:
        for i in range(len(data[split_])):
            for field in ['PrimaryGleason', 'SecondaryGleason', 'TertiaryGleason']:
                if data[split_][i]['labels'][field] == 0:
                    data[split_][i]['labels'][field] = 'null'
    return data

# Read in data
path = f"{base_dir}/data/splits/prostate.json"
data = readJson(path)

# Clean reports
data = cleanSplit(data, stripChars)
data['dev_test'] = cleanReports(data['dev_test'], stripChars)

data = fixLabel(data)
data = fixProstateLabels(data)

# Tokenizer
if args['model_type'] == 'bert':
    bert_path = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
elif args['model_type'] == 'medical_bert':
    bert_path = f"{base_dir}/models/pretrained/bert_pretrain_output_all_notes_150000/"
    tokenizer = BertTokenizer.from_pretrained(bert_path, local_files_only=True)
elif args['model_type'] == 'medical_bert':
    bert_path = f"{base_dir}/models/pretrained/bert_pretrain_output_all_notes_150000/"
    tokenizer = BertTokenizer.from_pretrained(bert_path, local_files_only=True)
elif args['model_type'] == 'pubmed_bert':
    bert_path = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
    tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
elif args['model_type'] == 'pubmed_bert_full':
    bert_path = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
    tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
elif args['model_type'] == 'biobert':
    bert_path = "dmis-lab/biobert-v1.1"
    tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
elif args['model_type'] == 'clinical_biobert':
    bert_path = "emilyalsentzer/Bio_ClinicalBERT"
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
elif args['model_type'] == 'tnlr':
    vocab_file = f'{code_dir}/turing/src/tnlr/tokenizer/tnlr-uncased-vocab.txt'
    tokenizer = load_tnlr_tokenizer(vocab_file)
    
train_documents = [extract_synoptic(patient['document'].lower(), tokenizer) for patient in data['train']]
val_documents = [extract_synoptic(patient['document'].lower(), tokenizer) for patient in data['val']]
test_documents = [extract_synoptic(patient['document'].lower(), tokenizer) for patient in data['test']]

kwargs = Namespace(**args)

Calling TuringNLRv3Tokenizer.from_pretrained() with the path to a single file or url is deprecated


# Train

In [2]:
fields = ['PrimaryGleason', 'SecondaryGleason', 'MarginStatusNone', 'SeminalVesicleNone']

for field in fields:
    print(field)
    # Set output directories
    args['output_dir'] = f"{base_dir}/output/frozen/{args['model_type']}_{args['run']}/{field}"
    args['cache_dir'] = f"{base_dir}/output/frozen/{args['model_type']}_{args['run']}/{field}"
    
    kwargs = Namespace(**args)

    # Create datasets
    train_documents = [extract_synoptic(patient['document'].lower(), tokenizer) for patient in data['train']]
    train_labels = [patient['labels'][field] for patient in data['train']]

    val_documents = [extract_synoptic(patient['document'].lower(), tokenizer) for patient in data['val']]
    val_labels = [patient['labels'][field] for patient in data['val']]

    test_documents = [extract_synoptic(patient['document'].lower(), tokenizer) for patient in data['test']]
    test_labels = [patient['labels'][field] for patient in data['test']]

    le = preprocessing.LabelEncoder()
    le.fit(train_labels)

    # Handle new class
    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    le_dict = {str(key):le_dict[key] for key in le_dict}
    
    for label in val_labels + test_labels:
        if str(label) not in le_dict:
            le_dict[str(label)] = len(le_dict)

    train_labels = [le_dict[str(label)] for label in train_labels]
    val_labels = [le_dict[str(label)] for label in val_labels]
    test_labels = [le_dict[str(label)] for label in test_labels]
        
    # Train model
    model = BertForSequenceClassification.from_pretrained(bert_path, num_labels=len(le_dict))

    documents_full = train_documents + val_documents + test_documents
    patients_full = data['train'] + data['val'] + data['test']

    p_test = len(test_labels)/len(patients_full)
    p_val = len(val_labels)/(len(train_labels) + len(val_labels))

    train_documents, test_documents, train_patients, test_patients = train_test_split(documents_full, 
                                                                                  patients_full, 
                                                                                  test_size= p_test,
                                                                                  random_state=args['run'])

    train_documents, val_documents, train_patients, val_patients = train_test_split(train_documents, 
                                                                                  train_patients, 
                                                                                  test_size= p_val,
                                                                                  random_state=args['run'])


    train_labels = [le_dict[str(patient['labels'][field])] for patient in train_patients]
    val_labels = [le_dict[str(patient['labels'][field])] for patient in val_patients]
    test_labels = [le_dict[str(patient['labels'][field])] for patient in test_patients]
        
    class_weights = make_weights_for_balanced_classes(train_labels, len(train_labels))
    
    train_dataset = path_dataset(train_documents, train_labels, model, tokenizer)
    val_dataset = path_dataset(val_documents, val_labels, model, tokenizer)
    test_dataset = path_dataset(test_documents, test_labels, model, tokenizer)
    
    train(kwargs, train_dataset, val_dataset, model, tokenizer, class_weights)

    del model

PrimaryGleason


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

{"learning_rate": 2.4453024453024455e-07, "loss": 1.8745895171165465, "step": 50}
{"learning_rate": 4.890604890604891e-07, "loss": 1.8913129878044128, "step": 100}
{"learning_rate": 7.335907335907336e-07, "loss": 1.9214560556411744, "step": 150}
{"learning_rate": 9.781209781209782e-07, "loss": 1.8696925568580627, "step": 200}
{"learning_rate": 1.2226512226512227e-06, "loss": 1.8906389379501343, "step": 250}
{'acc': 0.27079303675048355}
{"eval_acc": 0.27079303675048355, "learning_rate": 1.2666666666666667e-06, "loss": 0.3205408239364624, "step": 259}
/wynton/protected/project/pathnlp/path_nlp_turing/output/frozen/pubmed_bert_0/PrimaryGleason/epoch-1


Epoch:   3%|▎         | 1/30 [03:06<1:30:11, 186.61s/it]

...

{"learning_rate": 5.746460746460746e-07, "loss": 0.6547416591644287, "step": 7300}
{"learning_rate": 5.135135135135135e-07, "loss": 0.686141391992569, "step": 7350}
{"learning_rate": 4.5238095238095237e-07, "loss": 0.6728278708457947, "step": 7400}
{"learning_rate": 3.9124839124839125e-07, "loss": 0.6816840946674347, "step": 7450}
{"learning_rate": 3.301158301158301e-07, "loss": 0.6770962882041931, "step": 7500}
{'acc': 0.5783365570599613}
{"eval_acc": 0.5783365570599613, "learning_rate": 3.1666666666666667e-07, "loss": 0.1504865002632141, "step": 7511}
/wynton/protected/project/pathnlp/path_nlp_turing/output/frozen/pubmed_bert_0/SeminalVesicleNone/epoch-29


Epoch:  97%|█████████▋| 29/30 [1:22:32<02:50, 170.74s/it]

{"learning_rate": 2.68983268983269e-07, "loss": 0.5232982003688812, "step": 7550}
{"learning_rate": 2.0785070785070785e-07, "loss": 0.659570324420929, "step": 7600}
{"learning_rate": 1.4671814671814672e-07, "loss": 0.6817832851409912, "step": 7650}
{"learning_rate": 8.558558558558559e-08, "loss": 0.6819139671325684, "step": 7700}
{"learning_rate": 2.4453024453024453e-08, "loss": 0.6735980486869813, "step": 7750}
{'acc': 0.5802707930367504}
{"eval_acc": 0.5802707930367504, "learning_rate": 0.0, "loss": 0.2721320569515228, "step": 7770}
/wynton/protected/project/pathnlp/path_nlp_turing/output/frozen/pubmed_bert_0/SeminalVesicleNone/epoch-30


Epoch: 100%|██████████| 30/30 [1:25:23<00:00, 170.78s/it]

Best performance {'acc': 0.7021276595744681, 'checkpoint': 'epoch-1'}


# Load Model

In [2]:
def evaluate(eval_dataloader, args, model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_task_names = (
        ("mnli", "mnli-mm") if args.task_name == "mnli" else (args.task_name,)
    )

    results = {}
   
    # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "labels": batch[3],
            }
            if args.model_type != "distilbert":
                inputs["token_type_ids"] = (
                    batch[2] if args.model_type in ["bert", "xlnet"] else None
                )  # XLM, DistilBERT and RoBERTa don't use segment_ids
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(
                out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0
            )

    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds, axis=1)
    result = compute_metrics('mnli', preds, out_label_ids)
    results.update(result)
    
    results = {'macro': np.round(f1_score(out_label_ids, preds, average='macro'), 3),
               'micro': np.round(f1_score(out_label_ids, preds, average='micro'), 3)}

    return results, out_label_ids, preds

In [32]:
fields = ['PrimaryGleason', 'SecondaryGleason', 'MarginStatusNone', 'SeminalVesicleNone']
results = {field: {'macro': [], 'micro': []} for field in fields}
args['model_type'] = 'biobert'

for i in range(1):
    args['run'] = i
    
    for field in fields[:1]:

        # Create datasets
        train_labels = [patient['labels'][field] for patient in data['train']]
        val_labels = [patient['clean_labels'][field] for patient in data['val']]
        test_labels = [patient['clean_labels'][field] for patient in data['test']]

        le = preprocessing.LabelEncoder()
        le.fit(train_labels)

        # Handle new class
        le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        le_dict = {str(key):le_dict[key] for key in le_dict}

        for label in val_labels + test_labels:
            if str(label) not in le_dict:
                le_dict[str(label)] = len(le_dict)

        documents_full = train_documents + val_documents + test_documents
        labels_full = train_labels + val_labels + test_labels

        p_test = len(test_labels)/len(labels_full)
        p_val = len(val_labels)/(len(train_labels) + len(val_labels))

        train_docs, test_docs, train_labels, test_labels = train_test_split(documents_full, 
                                                                                      labels_full, 
                                                                                      test_size= p_test,
                                                                                      random_state=args['run'])

        train_docs, val_docs, train_labels, val_labels = train_test_split(train_docs, 
                                                                                      train_labels, 
                                                                                      test_size= p_val,
                                                                                      random_state=args['run'])

        train_labels = [le_dict[str(label)] for label in train_labels]
        val_labels = [le_dict[str(label)] for label in val_labels]
        test_labels = [le_dict[str(label)] for label in test_labels]
 
        best_performance = readJson(f"{base_dir}/path_nlp_turing/output/fine_tuning/{args['model_type']}_0/{field}/best_performance.json")
        checkpoint_file = f"{base_dir}/path_nlp_turing/output/fine_tuning/{args['model_type']}_0/{field}/{best_performance['checkpoint']}"
        config_file = f"{base_dir}/path_nlp_turing/output/fine_tuning/{args['model_type']}_0/{field}/save_output/config.json"
            
        if args['model_type'] != 'tnlr':
            model = BertForSequenceClassification.from_pretrained(checkpoint_file, num_labels=len(le_dict))
        else:
            model = load_tnlr_base(checkpoint_file, config_file, model_type='tnlrv3_classification', num_labels=len(le_dict))
        
        model = model.cpu()

        test_dataset = path_dataset(test_docs, test_labels, model, tokenizer)
        test_dataloader = DataLoader(test_dataset, batch_size=kwargs.per_gpu_train_batch_size)
        output, labels, preds = evaluate(test_dataloader, kwargs, model, tokenizer, prefix="")
        results[field]['micro'].append(output['micro'])
        results[field]['macro'].append(output['macro'])
                                    
        inv_le_dict = {v: k for k, v in le_dict.items()}

        preds = [inv_le_dict[pred] for pred in preds]
        labels = [inv_le_dict[label] for label in labels]
                                    
        y_actu = pd.Series(labels, name='Actual')
        y_pred = pd.Series(preds, name='Predicted')
        df_confusion = pd.crosstab(y_actu, y_pred)
                                    
        print(f"{field}_{args['run']}")
        print(df_confusion)
        print("\n")

        del model
        del test_dataloader

OSError: Can't load config for '/wynton/protected/project/pathnlp/path_nlp_turing/output/fine_tuning/biobert_0/PrimaryGleason/epoch-19'. Make sure that:

- '/wynton/protected/project/pathnlp/path_nlp_turing/output/fine_tuning/biobert_0/PrimaryGleason/epoch-19' is a correct model identifier listed on 'https://huggingface.co/models'

- or '/wynton/protected/project/pathnlp/path_nlp_turing/output/fine_tuning/biobert_0/PrimaryGleason/epoch-19' is the correct path to a directory containing a config.json file



In [25]:
for field in fields:
    print(field)
    print(results[field][0])

PrimaryGleason


KeyError: 'PrimaryGleason'